In [1]:
import bittensor as bt
import storage
import redis
import base64
import json
import sys
import os

bt.trace()

db = redis.Redis(host="localhost", port=6379, db=0)
db

Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

In [2]:
# create a wallet instance that must be registered on the network
wallet = bt.wallet(name="default", hotkey="default")
print(wallet)

# instantiate the metagraph
metagraph = bt.metagraph(
    netuid=22, network="test", sync=True, lite=False
)
print(metagraph)

# Grab the axon you're serving
axon = metagraph.axons[6]
print(axon)

# Create a Dendrite instance to handle client-side communication.
d = bt.dendrite(wallet=wallet)
print(d)

wallet(default, default, ~/.bittensor/wallets/)
metagraph(netuid:22, n:11, block:646419, network:test)
AxonInfo( /ipv4/149.137.225.62:10004, 5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU, 5GeYLB44QY9wcqJmFZvJW8D3EYPDaJGSgGfkbJVxUbkVcU7C, 620 )
dendrite(5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH)


In [18]:
axon

AxonInfo( /ipv4/149.137.225.62:10004, 5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU, 5GeYLB44QY9wcqJmFZvJW8D3EYPDaJGSgGfkbJVxUbkVcU7C, 620 )

In [3]:
# Setup CRS for initial round of validation
g, h = storage.shared.ecc.setup_CRS(curve="P-256")

# Make a bytes string to test the miner
data = b"this is a random bytestring, long enough to be chunked into segments and reconstructed at the end"

# Encrypt the data
encrypted_data, encryption_payload = storage.validator.encryption.encrypt_data(
    data,
    wallet,
)
encrypted_data

b'\xdd\xd7sv\xa2a\x93p\x1e\xc2\xcc\x8aYR\x8c`!\xfc\xa8\x83\x8d\xdai\x80.\x9fk^\xbd\xc7\x1b^<x\xc1)/\xd4$|Gp\x14\x93,Wc\xf6\xc6~*\x15\xf0\x15qE\xb9\xdb\x8eO:\xb6K{%+\t\x91\x9f\xb8\xe8\xd8+\xb4\x94]\xb1\xbd\x9c2&\xcch\x86C\xeb\x92y\xd5\xce\xca\x9e\x84\xc7\xc1LA'

In [4]:
data_hash = storage.shared.ecc.hash_data(encrypted_data)
print("data_hash:", data_hash)

encrypted_data

data_hash: 114118800427053723412699299725425869363133103567887688833379719181721603644456


b'\xdd\xd7sv\xa2a\x93p\x1e\xc2\xcc\x8aYR\x8c`!\xfc\xa8\x83\x8d\xdai\x80.\x9fk^\xbd\xc7\x1b^<x\xc1)/\xd4$|Gp\x14\x93,Wc\xf6\xc6~*\x15\xf0\x15qE\xb9\xdb\x8eO:\xb6K{%+\t\x91\x9f\xb8\xe8\xd8+\xb4\x94]\xb1\xbd\x9c2&\xcch\x86C\xeb\x92y\xd5\xce\xca\x9e\x84\xc7\xc1LA'

In [5]:
# Convert to base64 for compactness
b64_encrypted_data = base64.b64encode(encrypted_data).decode("utf-8")
b64_encrypted_data

'3ddzdqJhk3AewsyKWVKMYCH8qION2mmALp9rXr3HG148eMEpL9QkfEdwFJMsV2P2xn4qFfAVcUW5245POrZLeyUrCZGfuOjYK7SUXbG9nDImzGiGQ+uSedXOyp6Ex8FMQQ=='

In [6]:
# Create the synapse to store it
synapse = storage.protocol.Store(
    encrypted_data=b64_encrypted_data,
    data_hash=data_hash,
    curve="P-256",
    g=storage.shared.ecc.ecc_point_to_hex(g),
    h=storage.shared.ecc.ecc_point_to_hex(h),
    seed=os.urandom(32).hex(),
)
synapse

Store(required_hash_fields=['curve', 'g', 'h', 'seed', 'randomness', 'commitment', 'signature', 'commitment_hash'], encrypted_data='3ddzdqJhk3AewsyKWVKMYCH8qION2mmALp9rXr3HG148eMEpL9QkfEdwFJMsV2P2xn4qFfAVcUW5245POrZLeyUrCZGfuOjYK7SUXbG9nDImzGiGQ+uSedXOyp6Ex8FMQQ==', curve='P-256', g='33393732373737373335343135353434393434333937303030333930303239343833363037383833323730373739323134333133313939333332303237303136353235313432363930393338302c39303730333532333539323135363236383231393836363636373031383432313336373139353132383137323232303031393236383437333432363630343531333130353030393736323538', h='3131333239373038323133323536353035353230393532363430313032303736363837343931363038333730363835323839393735383035303935393636323837353539313138313332363432322c3731383633383634333634333632343639373934333233313335343030373939373330313138393835393431393238373330323033353133353738383231383332393531393231363034393939', seed='413c826262a564e537e9175b5cc54dc8a6f8d40f71320875b8233cd65c297ca4', randomness=No

In [19]:
ping = bt.Synapse()
bt.trace()
# Send a request to the Axon using the Dendrite, passing in a Synapse 
# instance with the data_hash field. 
responses = await d(
    [axon],
    ping,
    deserialize=False,
)
responses

2023-11-16 19:53:51.821 |      DEBUG       | dendrite | --> | 3159 B | Synapse | 5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU | 149.137.225.62:10004 | 0 | Success
2023-11-16 19:53:51.842 |      DEBUG       | dendrite | <-- | 4044 B | Synapse | 5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU | 149.137.225.62:10004 | 401 | Not Verified with error: Signature mismatch with 11073186306859948.5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH.5H1CaMTf6XvzSYLcLe4AhnXwttQ4z3ry6fy9XzS9wrxV5vFA.c2153ac0-84b8-11ee-8804-3ceceff2faf8.a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a and 0x6c6879815c3ab482c774a101b89d4c1ecd3d8047aa7f1530a4cab600b9cc040b631ad65473005f57feb1dd4cfa683137a08eb23b6587a82dc1d955f5b4d0648b


[Synapse()]

In [15]:
# Send a request to the Axon using the Dendrite, passing in a Synapse 
# instance with the data_hash field. 
responses = await d(
    [axon],
    synapse,
    deserialize=False,
)
responses

2023-11-16 19:43:07.127 |      DEBUG       | dendrite | --> | 5021 B | Store | 5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU | 149.137.225.62:10004 | 0 | Success
2023-11-16 19:43:07.168 |      DEBUG       | dendrite | <-- | 5922 B | Store | 5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU | 149.137.225.62:10004 | 401 | Not Verified with error: Signature mismatch with 11072541613459067.5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH.5H1CaMTf6XvzSYLcLe4AhnXwttQ4z3ry6fy9XzS9wrxV5vFA.4c513cc6-84b8-11ee-9fff-3ceceff2faf8.5a6b75ebbd1630c9bae2d01183a8b4f381cd85fef04d09b97918effa05562b45 and 0x560896ae7a28146471130448126bb8c8b9ec9ec1480ffcf83c6ccd42ca7d5e044c1661296c6726ad36e582fc5132e1794ebe6f22e84772053e3bba53ad313b87


[Store(required_hash_fields=['curve', 'g', 'h', 'seed', 'randomness', 'commitment', 'signature', 'commitment_hash'], encrypted_data='zge84tNAjyYzDBugZtk7m6SQu92O1Fw+7l4tABHkO1xoWQg0+ZogXSoNupXnnf4bA13iKtmZfE7IFLSJXQxlxDNmQgm6c28xHlMUAFReBD3LjctWiXjv7e91HrsyFCCXXw==', curve='P-256', g='38303130353035383830303530393633323231373337313435303734353631383135383133383633303033393338363732393030353834303132363132333536353937393930393639353930372c313130353430313238353737343232393933353930343536373632363030373332303733383939323333393735323136353437333431313539353635343238323438323134303337343831373135', h='36313733333739303537303631363930323433353136343739373331353130363438383537373130313634393935343039393039333639343734333738313836363336313334353230363138372c3633393232383234323936313030343830323539373735343130393834343631373834353137353932393034353732383431333831333937363036343532313033333338393331303334363730', seed='73707881062fee6bf58c2926933e1d760744e8b6e1af3ec929eb056d51d9d552', randomness

In [20]:
resp = responses[0]
resp.dict()

{'name': 'Synapse',
 'timeout': 12.0,
 'total_size': 4044,
 'header_size': 0,
 'dendrite': {'status_code': 401,
  'status_message': 'Not Verified with error: Signature mismatch with 11073186306859948.5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH.5H1CaMTf6XvzSYLcLe4AhnXwttQ4z3ry6fy9XzS9wrxV5vFA.c2153ac0-84b8-11ee-8804-3ceceff2faf8.a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a and 0x6c6879815c3ab482c774a101b89d4c1ecd3d8047aa7f1530a4cab600b9cc040b631ad65473005f57feb1dd4cfa683137a08eb23b6587a82dc1d955f5b4d0648b',
  'process_time': 0.021933555603027344,
  'ip': '127.0.0.1',
  'port': 35890,
  'version': 620,
  'nonce': 11073186306859948,
  'uuid': 'c2153ac0-84b8-11ee-8804-3ceceff2faf8',
  'hotkey': '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH',
  'signature': '0x6c6879815c3ab482c774a101b89d4c1ecd3d8047aa7f1530a4cab600b9cc040b631ad65473005f57feb1dd4cfa683137a08eb23b6587a82dc1d955f5b4d0648b'},
 'axon': {'status_code': 401,
  'status_message': 'Not Verified with erro

In [10]:
# Store necessary data on the validator side
response_storage = {
    "size": sys.getsizeof(encrypted_data),
    "prev_seed": synapse.seed,
    "commitment_hash": resp.commitment_hash,  # contains the seed
    # these will be private to the validator, not stored in decentralized GUN db
    # just here now for testing purposes
    "encryption_payload": encryption_payload
}
response_storage

{'size': 130,
 'prev_seed': '09267afc92212859ffbeb8d7be0bcec6d8d799dc4d3cf616087dbf3a7e5d54a3',
 'commitment_hash': None,
 'encryption_payload': '{"nonce": "3bc21783a90e2555951bf5a57fac20d45a07e3e4bfc17bda", "ciphertext": "9f7bb2f35b212123af59b0f4a204d86b03a78be1607fc3147a06eff51f059092cb09147a730cffc7d8f648f3a01b0198f5728f1ddfae80a086ae5520c5654b506126a13d052d05d11d45d065928b18961140662b46275577fbfbb8196c41ede3fa5d005bdbdcd9330f6f798a103b4bf1199320cfcea208ed2fe22c00be402df380a7ca532cb983534293d09b105227510eda18abee7de1061708b20222d39490ad7dc60e8ac2404561962d92c685a656f160536931d1a5"}'}

In [11]:
# Store in the database according to the data hash and the miner hotkey
key = f"{data_hash}.{resp.axon.hotkey}"
print("validator storage key:", key)
db.set(key, json.dumps(response_storage).encode())

validator storage key: 45523629128671725334410933537370912552436178063014474958404965724993446848727.5EJ2bC6t77aGaYcQzBkMsFKKiYSNedNq6ugEJPiW7RvtQsw7


True

In [12]:
storage.validator.verify.verify_store_with_seed(resp)

2023-11-16 19:17:44.201 |      ERROR       | Initial commitment hash does not match hash(data + seed)
2023-11-16 19:17:44.201 |      ERROR       | synapse: {'status_code': 503, 'status_message': 'Service at 23.228.172.18:9999/Store unavailable.', 'process_time': None, 'ip': '149.137.225.62', 'port': None, 'version': 620, 'nonce': 11071018589697251, 'uuid': 'c911ac2c-84b4-11ee-b2f1-3ceceff2faf8', 'hotkey': '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH', 'signature': '0x94aa6c7b4865abdd1a57c39b43a967a5229dc6620fd2887b1c4408e430366f1f8ddaad25c10a6f27b393c481ac353e6f64bc8ebd9047f58a1310e332755e6a84'}


False

In [13]:
# Setup a challenge to the miner with a new CRS and on data we just stored
g, h = storage.shared.ecc.setup_CRS(curve="P-256")
challenge = storage.protocol.Challenge(
    challenge_hash=data_hash,
    g=storage.shared.ecc.ecc_point_to_hex(g),
    h=storage.shared.ecc.ecc_point_to_hex(h),
    curve="P-256",
    seed=os.urandom(32).hex(),
    challenge_index=1,
    chunk_size=storage.validator.utils.get_random_chunksize(sys.getsizeof(data)) // 4, # bytes
)
challenge

Challenge(required_hash_fields=['commitment_hash', 'commitment_proof', 'commitment', 'data_chunk', 'randomness', 'merkle_proof', 'merkle_root'], challenge_hash='45523629128671725334410933537370912552436178063014474958404965724993446848727', challenge_index=1, chunk_size=74, g='3131303435303736383734393135383738313834323131373231373737343637363735333539383139373930333031343637323634363134343633393936393935383832383739383832383736312c313032383035373634303433343134353630303439333733333738343135343837333030323438373734383637393531363633383735393631343630383039373339323732323039393035373438', h='38383836343532383831383336373931323032373937333730363834383733383038383430393533333434353834333230353533353731373830313333303736363438343238373339363134382c3230393539353537373230393135313937343932363436393337393039363931343334313337363539363736303234353937323534313838303439323931353037323930393530353230333933', curve='P-256', seed='e3fbb74a319ba941ccb8fdfbf862e20976d44c22d4698b250fa502e49a3cb745', c

In [14]:
# Send the challenge to the miner
responses = d.query(
    [metagraph.axons[13]],
    challenge,
    deserialize=False,
)
responses[0]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 # Send the challenge to the miner                                                            │
│   2 responses = d.query(                                                                         │
│ ❱ 3 │   [metagraph.axons[13]],                                                                   │
│   4 │   challenge,                                                                               │
│   5 │   deserialize=False,                                                                       │
│   6 )                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: list index out of range

In [15]:
# Verify the challenge
storage.validator.verify.verify_challenge_with_seed(responses[0])

2023-11-16 19:17:44.699 |      ERROR       | Missing commitment hash or proof for synapse: {'status_code': 503, 'status_message': 'Service at 23.228.172.18:9999/Store unavailable.', 'process_time': None, 'ip': '149.137.225.62', 'port': None, 'version': 620, 'nonce': 11071018589697251, 'uuid': 'c911ac2c-84b4-11ee-b2f1-3ceceff2faf8', 'hotkey': '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH', 'signature': '0x94aa6c7b4865abdd1a57c39b43a967a5229dc6620fd2887b1c4408e430366f1f8ddaad25c10a6f27b393c481ac353e6f64bc8ebd9047f58a1310e332755e6a84'}.


False

In [16]:
# Now retrieve the data we stored and successfully challenged
syn = storage.protocol.Retrieve(
    data_hash=data_hash,
    seed=os.urandom(32).hex(),
)
syn

Retrieve(required_hash_fields=['data', 'data_hash', 'seed', 'commtiment_proof', 'commitment_hash'], data_hash='45523629128671725334410933537370912552436178063014474958404965724993446848727', seed='71d86a6672f385fd521968937d3fef9ad8db67a4bc9d010b1e77133fdca683d5', data=None, commitment_hash=None, commitment_proof=None)

In [17]:
# Send a request to the Axon using the Dendrite, passing in a Synapse 
# instance with the data_hash field. 
responses = await d(
    [axon],
    syn,
    deserialize=False,
)
responses 

2023-11-16 19:17:44.712 |      DEBUG       | dendrite | --> | 4100 B | Retrieve | 5EJ2bC6t77aGaYcQzBkMsFKKiYSNedNq6ugEJPiW7RvtQsw7 | 23.228.172.18:9999 | 0 | Success
2023-11-16 19:17:44.758 |      DEBUG       | dendrite | <-- | 4256 B | Retrieve | 5EJ2bC6t77aGaYcQzBkMsFKKiYSNedNq6ugEJPiW7RvtQsw7 | 23.228.172.18:9999 | 503 | Service at 23.228.172.18:9999/Retrieve unavailable.


[Retrieve(required_hash_fields=['data', 'data_hash', 'seed', 'commtiment_proof', 'commitment_hash'], data_hash='45523629128671725334410933537370912552436178063014474958404965724993446848727', seed='71d86a6672f385fd521968937d3fef9ad8db67a4bc9d010b1e77133fdca683d5', data=None, commitment_hash=None, commitment_proof=None)]

In [18]:
# extract the base64 encoded string of the data
resp = responses[0]
resp.data

In [19]:
# decode from base64 to bytes
decoded_data = base64.b64decode(resp.data)
decoded_data

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # decode from base64 to bytes                                                                │
│ ❱ 2 decoded_data = base64.b64decode(resp.data)                                                   │
│   3 decoded_data                                                                                 │
│   4                                                                                              │
│                                                                                                  │
│ /home/phil/miniconda3/envs/storage/lib/python3.10/base64.py:80 in b64decode                      │
│                                                                                                  │
│    77 │   to the padding check.  If validate is True, these non-alphabet characters              │
│    78 │   in the input result in a binascii.Error.                                               │
│    79 │   """                                                                                    │
│ ❱  80 │   s = _bytes_from_decode_data(s)                                                         │
│    81 │   if altchars is not None:                                                               │
│    82 │   │   altchars = _bytes_from_decode_data(altchars)                                       │
│    83 │   │   assert len(altchars) == 2, repr(altchars)                                          │
│                                                                                                  │
│ /home/phil/miniconda3/envs/storage/lib/python3.10/base64.py:45 in _bytes_from_decode_data        │
│                                                                                                  │
│    42 │   try:                                                                                   │
│    43 │   │   return memoryview(s).tobytes()                                                     │
│    44 │   except TypeError:                                                                      │
│ ❱  45 │   │   raise TypeError("argument should be a bytes-like object or ASCII "                 │
│    46 │   │   │   │   │   │   "string, not %r" % s.__class__.__name__) from None                 │
│    47                                                                                            │
│    48                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: argument should be a bytes-like object or ASCII string, not 'NoneType'

In [20]:
# decrypt back to the original file content bytes
storage.validator.encryption.decrypt_data(
    decoded_data,
    response_storage['encryption_payload'],
    wallet=wallet,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 # decrypt back to the original file content bytes                                            │
│   2 storage.validator.encryption.decrypt_data(                                                   │
│ ❱ 3 │   decoded_data,                                                                            │
│   4 │   response_storage['encryption_payload'],                                                  │
│   5 │   wallet=wallet,                                                                           │
│   6 )                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'decoded_data' is not defined

In [21]:
import torch

In [26]:
vpermits = metagraph.validator_permit
vpermits

Parameter containing:
tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True, False,
        False])

In [27]:
vpermit_uids = [uid for uid, permit in enumerate(vpermits) if permit]
vpermit_uids

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [28]:
vpermit_uids = torch.where(vpermits)[0]
vpermit_uids

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [29]:
# Exclude your own uid
vpermit_uids = vpermit_uids[
    vpermit_uids
    != metagraph.hotkeys.index(wallet.hotkey.ss58_address)
]
vpermit_uids

tensor([1, 2, 3, 4, 5, 6, 7, 8])

In [35]:
metagraph.S[vpermit_uids]

tensor([3.9402e+01, 5.4623e+01, 2.0966e+02, 3.3365e+01, 3.5353e+04, 1.0174e+02,
        5.0871e+03, 0.0000e+00])

In [37]:
query_idxs = torch.where(
    metagraph.S[vpermit_uids] > 1000
)[0]
query_idxs

tensor([4, 6])

In [ ]:
query_uids = vpermit_uids[query_idxs]

In [ ]:
axons = [metagraph.axons[uid] for uid in query_uids]
query_uids